In [3]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd 

headers = {'User Agent':'Mozilla/5.0'}

players = [
    'kyrie-irving', 'derrick-williams-2', 'tristan-thompson', 'brandon-knight', 'kemba-walker', 
    'jimmer-fredette', 'klay-thompson','alec-burks', 'markieff-morris', 'marcus-morris', 'kawhi-leonard', 
    'nikola-vucevic', 'iman-shumpert', 'chris-singleton','tobias-harris', 'nolan-smith', 'kenneth-faried', 
    'reggie-jackson', 'marshon-brooks', 'jordan-hamilton', 'jajuan-johnson', 'norris-cole', 'cory-joseph', 
    'jimmy-butler', 'justin-harper', 'kyle-singler', 'shelvin-mack', 'tyler-honeycutt', 'trey-thompkins',
    'chandler-parsons', 'jon-leuer', 'darius-morris', 'malcolm-lee', 'charles-jenkins', 'josh-harrellson', 
    'travis-leslie', 'keith-benson', 'josh-shelby', 'lavoy-allen', 'jon-diebler', 'vernon-macklin', 
    'deandre-liggins', 'etwaun-moore', 'isaiah-thomas']   

player_stats = []
playerlist = []

for player in players:
    try:
        if player[-1].isdigit():
            url = f'https://www.sports-reference.com/cbb/players/{player}.html'
        else:
            url = f'https://www.sports-reference.com/cbb/players/{player}-1.html'
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        header = [th.getText() for th in soup.findAll('tr', limit = 2)[0].findAll('th')]
        rows = soup.findAll('tr')
        player_stats.append([td.getText() for td in soup.find('tr', id ='players_per_game.2011')])
        table = pd.DataFrame(player_stats, columns = header) 
        for i in player_stats:
            if i[14] == '':
                i[14] = str(0.0)
        
        player = player.replace('-',' ').title()
        if player[-1].isdigit():
            player = player[:-2]
        if player[-2:] == 'jr':
            player = player.replace('jr', ' Jr')
        playerlist.append(player)
        
    except:
        continue
  
table.insert(0, "Name", playerlist)
table['MP'] = table['MP'].astype(float)
table['PTS'] = table['PTS'].astype(float)
table['AST'] = table['AST'].astype(float)
table['TRB'] = table['TRB'].astype(float)
table['BLK'] = table['BLK'].astype(float)
table['STL'] = table['STL'].astype(float)
table['3P'] = table['3P'].astype(float)
table['3PA'] = table['3PA'].astype(float)
table['TOV'] = table['TOV'].astype(float)
table['SOS'] = table['SOS'].astype(float)
table['PF'] = table['PF'].astype(float)
table['FT%'] = table['FT%'].astype(float)
table['3P%'] = table['3P%'].astype(float)

per40 = 40/table['MP']

table["Player Grade"] = ((table['PTS']*per40) + (table['TRB']*1.5*per40) + (table['AST']*2*per40) +
(table['BLK']*3*per40) + (table['STL']*3*per40) + (table['3P']*5*per40)+ (table['FT%']*7) + (table['SOS']) + 
(table['3P%']*10))  

table["Player Grade"] = table["Player Grade"]*1.15
table["Player Grade"] = (round(table["Player Grade"]))
table["Player Grade"]= table["Player Grade"].astype(int)
                        
table = table.sort_values(by= "Player Grade", ascending=False)
table.insert(4, 'SoS', table['SOS'])
table.reset_index(drop = True, inplace=True)
import numpy as np
table.index = np.arange(1, len(table)+1)
del table['SOS']
del table['Conf']
del table['ORB']
del table['DRB']
del table['GS']
del table['FG']
del table['FGA']
del table['FT']
del table['FTA']
del table['2P']
del table['2PA']
del table['2P%']
table

,Name,Season,School,SoS,G,MP,FG%,3P,3PA,3P%,FT%,TRB,AST,STL,BLK,TOV,PF,PTS,,Player Grade
1,Jimmer Fredette,2010-11,BYU,5.76,37,35.8,.452,3.4,8.5,0.396,0.894,3.4,4.3,1.3,0.0,3.5,1.3,28.9,,100
2,Kyrie Irving,2010-11,Duke,8.32,11,27.5,.529,1.6,3.5,0.462,0.901,3.4,4.3,1.5,0.5,2.5,2.1,17.5,,98
3,Klay Thompson,2010-11,Washington State,6.81,34,34.7,.436,2.9,7.2,0.398,0.838,5.2,3.7,1.6,0.9,3.4,2.9,21.6,,97
4,Marshon Brooks,2010-11,Providence,6.44,32,36.5,.483,2.1,6.2,0.340,0.772,7.0,2.5,1.5,1.2,3.1,2.5,24.6,,92
5,Kemba Walker,2010-11,UConn,10.44,41,37.6,.428,1.8,5.5,0.330,0.819,5.4,4.5,1.9,0.2,2.3,1.4,23.5,,91
6,Markieff Morris,2010-11,Kansas,7.62,38,24.4,.589,0.7,1.6,0.424,0.673,8.3,1.4,0.8,1.1,2.1,2.8,13.6,,91
7,Derrick Williams,2010-11,Arizona,7.99,38,30.0,.595,1.1,1.9,0.568,0.746,8.3,1.1,1.0,0.7,2.6,2.8,19.5,,90
8,Iman Shumpert,2010-11,Georgia Tech,5.68,31,32.0,.406,1.4,4.9,0.278,0.806,5.9,3.5,2.7,0.2,2.3,2.8,17.3,,86
9,Reggie Jackson,2010-11,Boston College,7.31,34,34.1,.503,2.1,5.0,0.420,0.796,4.3,4.5,1.1,0.5,2.4,1.7,18.2,,86
10,Nolan Smith,2010-11,Duke,8.32,37,34.0,.458,1.5,4.4,0.350,0.813,4.5,5.1,1.2,0.1,3.2,1.9,20.6,,86
